In [1]:
import sys
import os
from filecmp import cmp
import numpy as np
import pytest
import importlib
from astroquery.jplhorizons import Horizons
HEAD_DIR = os.path.dirname(os.path.realpath(os.getcwd()))
sys.path.append(os.path.join(HEAD_DIR))
from mpc_nbody import mpc_nbody, parse_input
importlib.reload(mpc_nbody)

<module 'mpc_nbody.mpc_nbody' from '/home/mikea/GitHub/mpc_nbody/mpc_nbody/mpc_nbody.py'>

In [2]:
DATA_DIR = os.path.join(HEAD_DIR, 'dev_data')
filenames = [os.path.join(DATA_DIR, file)
              for file in ['30101.eq0_horizons', '30102.eq0_horizons']]
au_km = 149597870.700  # This is now a definition
vector1 = [-2.093834952466475E+00, 1.000913720009255E+00, 4.197984954533551E-01, -4.226738336365523E-03, -9.129140909705199E-03, -3.627121453928710E-03]
vector2 = [-3.143563543369602e+00, 2.689063646113277E+00, 3.554211184881579E+00, -5.610620819862405e-03, -4.232958051824352E-03, -1.638364029313663E-03]
Parsed1 = parse_input.ParseElements(filenames[0], 'eq')
Parsed2 = parse_input.ParseElements(filenames[1], 'eq')

# Explicit Test Code Exists in mpc_nbody/tests

 - test_parse_input.py
 - test_run_nbody.py
 
 Run these using pytest

# Top-level: "mpc_nbody.NbodySim" - Parse, run, save, cheby.
 - Instantiation takes either no input (for manually feeding vectors later) or an input file in ele220 (not yet implemented) or OrbFit format, then calls the parser from parse_input. 
 - Calling the class optionally takes a tstart, list of vectors, timestep, time range. If a filename was supplied at instantiation, the class can be called with no input to simply integrate that orbit from the epoch of the orbit (although one might want to change the default tstep and trange). 
 - Both call a number of under-lying functionalities.

In [3]:
%%time
importlib.reload(mpc_nbody)
# First, let's initiate the class with an input file:
Sim = mpc_nbody.NbodySim(filenames[0], 'eq', save_parsed=True)
# Because save_parsed=True, the parsed orbit is saved to a holman_ic file:
!cat holman_ic
# (this is optional; the holman_ic file does not get used. 
# To demonstrate, let's delete it.)
!rm holman_ic
# Now run the integrator, by calling the object. 
Sim(tstep=20, trange=600, save_output=True)
# Because of the (again, optional) save_output=True, output was saved here:
!head simulation_states.dat
!wc simulation_states.dat
#This information is also all available inside the object now:
for item in Sim.__dict__ : print(item)

tstart 2456117.641933589
tstep +20.0
trange 600.
geocentric 0
state
-2.093834952466474e+00  1.000913720009255e+00  4.197984954533551e-01 
-4.226738336365523e-03 -9.129140909705197e-03 -3.627121453928710e-03 


#Input vectors: [-2.0938349524664743 1.0009137200092553 0.41979849545335507 -0.004226738336365523 -0.009129140909705197 -0.0036271214539287102]
#Input N_particles: 1
#Start time, timestep, time range: [2456117.641933589, 20, 600]
#Output N_times: 328
#Output N_particles: 1
#
#Time               x                  y                  z                   dx                    dy                    dz                  
2456117.641933589 -2.0938349524664743 1.0009137200092553 0.41979849545335507 -0.004226738336365523 -0.009129140909705197 -0.0036271214539287102 
2456118.7671847995 -2.0985611457606694 0.9906268479863111 0.41571107526570966 -0.004173477380689331 -0.00915450997156762 -0.003637765737845294 
2456121.2467474234 -2.1087634347251702 0.9678591498897638 0.4066622393235186 -0.00

# Lower-level functionalities
 - The above "NbodySim" usage uses some underlying functionality to perform the parsing, integration and storage of output.
 - Here we demonstrate some of the underlying functionality.

##### mpc_nbody.NbodySim.\_\_init__
 - With no arguments to set up an empty object. 

In [4]:
Sim = mpc_nbody.NbodySim()
Sim.__dict__

Keywords 'input_file' and/or 'filetype' missing; initiating empty object.


{'pparticle': None,
 'geocentric': False,
 'input_vectors': None,
 'input_n_particles': None,
 'output_times': None,
 'output_vectors': None,
 'output_n_times': None,
 'output_n_particles': None,
 'time_parameters': None}

##### mpc_nbody.NbodySim.\_\_call__
 - Runs the N-body integrator. Can be done in a few different ways.

In [5]:
#Method 1, using a list containing the 6 elements of an object
importlib.reload(mpc_nbody)
Sim = mpc_nbody.NbodySim()
Sim(vectors = vector1,
    tstart=2456117.5, tstep=20, trange=600)
_ = [print(k, np.shape(Sim.__dict__[k]), type(Sim.__dict__[k]))
     for k in Sim.__dict__]

Keywords 'input_file' and/or 'filetype' missing; initiating empty object.


pparticle () <class 'NoneType'>
geocentric () <class 'bool'>
input_vectors (6,) <class 'numpy.ndarray'>
input_n_particles () <class 'int'>
output_times (328,) <class 'numpy.ndarray'>
output_vectors (328, 1, 6) <class 'numpy.ndarray'>
output_n_times () <class 'int'>
output_n_particles () <class 'int'>
time_parameters (3,) <class 'list'>


In [6]:
#Method 2, using a list containing the 6 elements of multiple objects
importlib.reload(mpc_nbody)
Sim = mpc_nbody.NbodySim()
Sim(vectors = vector1 + vector2,
    tstart=2456117.5, tstep=20, trange=600)
_ = [print(k, np.shape(Sim.__dict__[k]), type(Sim.__dict__[k]))
     for k in Sim.__dict__]

Keywords 'input_file' and/or 'filetype' missing; initiating empty object.


pparticle () <class 'NoneType'>
geocentric () <class 'bool'>
input_vectors (12,) <class 'numpy.ndarray'>
input_n_particles () <class 'int'>
output_times (328,) <class 'numpy.ndarray'>
output_vectors (328, 2, 6) <class 'numpy.ndarray'>
output_n_times () <class 'int'>
output_n_particles () <class 'int'>
time_parameters (3,) <class 'list'>


In [7]:
#Method 3, parse_input.ParseElements object
importlib.reload(mpc_nbody)
Sim = mpc_nbody.NbodySim()
Sim(vectors = Parsed1, tstart=Parsed1.time.tdb.jd, tstep=20, trange=600)
_ = [print(k, np.shape(Sim.__dict__[k]), type(Sim.__dict__[k]))
     for k in Sim.__dict__]

Keywords 'input_file' and/or 'filetype' missing; initiating empty object.


pparticle () <class 'NoneType'>
geocentric () <class 'bool'>
input_vectors (6,) <class 'numpy.ndarray'>
input_n_particles () <class 'int'>
output_times (328,) <class 'numpy.ndarray'>
output_vectors (328, 1, 6) <class 'numpy.ndarray'>
output_n_times () <class 'int'>
output_n_particles () <class 'int'>
time_parameters (3,) <class 'list'>


In [8]:
#Method 4, a list of parse_input.ParseElements objects
importlib.reload(mpc_nbody)
Sim = mpc_nbody.NbodySim()
Sim(vectors = [Parsed1, Parsed2], tstart=Parsed1.time.tdb.jd, tstep=20, trange=600)
_ = [print(k, np.shape(Sim.__dict__[k]), type(Sim.__dict__[k]))
     for k in Sim.__dict__]

Keywords 'input_file' and/or 'filetype' missing; initiating empty object.



pparticle () <class 'NoneType'>
geocentric () <class 'bool'>
input_vectors (12,) <class 'numpy.ndarray'>
input_n_particles () <class 'int'>
output_times (328,) <class 'numpy.ndarray'>
output_vectors (328, 2, 6) <class 'numpy.ndarray'>
output_n_times () <class 'int'>
output_n_particles () <class 'int'>
time_parameters (3,) <class 'list'>


In [9]:
#Method 5, have NbodySim do stuff automatically 
#(only implemented for one object)
importlib.reload(mpc_nbody)
Sim = mpc_nbody.NbodySim(filenames[0], 'eq')
# This essentially stores the parsed elements within a pparticle atribute in Sim
Sim(tstep=20, trange=600)
_ = [print(k, np.shape(Sim.__dict__[k]), type(Sim.__dict__[k]))
     for k in Sim.__dict__]



pparticle () <class 'mpc_nbody.parse_input.ParseElements'>
geocentric () <class 'bool'>
input_vectors (6,) <class 'numpy.ndarray'>
input_n_particles () <class 'int'>
output_times (328,) <class 'numpy.ndarray'>
output_vectors (328, 1, 6) <class 'numpy.ndarray'>
output_n_times () <class 'int'>
output_n_particles () <class 'int'>
time_parameters (3,) <class 'list'>


##### mpc_nbody.NbodySim.save_output()
 - Saves the output to a file (filename is optional argument).

In [10]:
importlib.reload(mpc_nbody)
Sim = mpc_nbody.NbodySim(filenames[0], 'eq')
Sim(tstep=20, trange=600)
Sim.save_output(output_file='notebook_save.dat')
!head notebook_save.dat



#Input vectors: [-2.0938349524664743 1.0009137200092553 0.41979849545335507 -0.004226738336365523 -0.009129140909705197 -0.0036271214539287102 ]
#Input N_particles: 1
#Start time, timestep, time range: [2456117.641933589, 20, 600]
#Output N_times: 328
#Output N_particles: 1
#
#Time               x                  y                  z                   dx                    dy                    dz                  
2456117.641933589 -2.0938349524664743 1.0009137200092553 0.41979849545335507 -0.004226738336365523 -0.009129140909705197 -0.0036271214539287102 
2456118.7671847995 -2.0985611457606694 0.9906268479863111 0.41571107526570966 -0.004173477380689331 -0.00915450997156762 -0.003637765737845294 
2456121.2467474234 -2.1087634347251702 0.9678591498897638 0.4066622393235186 -0.0040554819245086 -0.009209585437161112 -0.003660894744530345 


##### mpc_nbody.run_nbody(input_vectors, tstart, tstep, trange)
 - The function that actually runs the N-body integrator. Called by NbodySim

In [11]:
importlib.reload(mpc_nbody)
(input_vectors, input_n_particles, output_times, output_vectors, output_n_times, output_n_particles
 ) = mpc_nbody.run_nbody(vector1 + vector2, tstart=2456117.5,
                         tstep=20, trange=600, geocentric=False)
np.shape(output_vectors)

(328, 2, 6)

##### mpc_nbody._fix_input(input)
 - Used to interpret what kind of input is given (6 element list, ParseElements object, list of ParseElements objects) and returns a standardized format (list containing 6 elements for each object). 
 - Not intended for user usage, but demonstrated here anyway. 

In [12]:
# Case 1: a single parse_input.ParseElements object
reparsed, nobj = mpc_nbody._fix_input(Parsed1)
for i in np.arange(nobj): print(f'object {i}', reparsed[0+i:6+i])
# Case 2: a list of parse_input.ParseElements objects
reparsed, nobj = mpc_nbody._fix_input([Parsed1, Parsed2])
for i in np.arange(nobj): print(f'object {i}', reparsed[0+i:6+i])
# Case 3: a list containing the 6-vector for an object
reparsed, nobj = mpc_nbody._fix_input(vector1)
for i in np.arange(nobj): print(f'object {i}', reparsed[0+i:6+i])
# Case 4: a list containing the 6-vectors of multiple objects as one long list.
# (this case is identical to the output)
reparsed, nobj = mpc_nbody._fix_input(vector1 + vector2)
for i in np.arange(nobj): print(f'object {i}', reparsed[0+i:6+i])


object 0 [-2.09383495  1.00091372  0.4197985  -0.00422674 -0.00912914 -0.00362712]


object 0 [-2.09383495  1.00091372  0.4197985  -0.00422674 -0.00912914 -0.00362712]
object 1 [ 1.00091372  0.4197985  -0.00422674 -0.00912914 -0.00362712 -3.14356354]

object 0 [-2.09383495  1.00091372  0.4197985  -0.00422674 -0.00912914 -0.00362712]

object 0 [-2.09383495  1.00091372  0.4197985  -0.00422674 -0.00912914 -0.00362712]
object 1 [ 1.00091372  0.4197985  -0.00422674 -0.00912914 -0.00362712 -3.14356354]
